In [1]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


pd.options.display.max_rows = 1000
pd.options.display.max_columns = 20

import warnings

# 경고 무시하기
warnings.filterwarnings("ignore")
# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'AppleGothic'# (위도우용)
from matplotlib import font_manager, rc

# AppleGothic 폰트 설정
rc('font', family='AppleGothic')

In [2]:
import pandas as pd
df = pd.read_csv('강남구전세데이터_마지막.csv')
df

,보증금(만원),건축년도,구분,전용면적(㎡),층,도로명주소_x,계약시작,계약종료,계약년월일,건축나이,...,행정구,행정구코드,스타벅스 개수_y,미쉐린 개수,맥도날드 개수,대규모점포 개수,전통시장 개수,역세권 개수,금리,유치원 개수
0,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,개포동,0,0,0,0,0,0,0,4.04,2
1,14333,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-10-01,2024-10-01,2022-10-05,32,...,개포동,0,0,0,0,0,0,0,4.82,2
2,17115,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,개포동,0,0,0,0,0,0,0,4.16,2
3,17640,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,개포동,0,0,0,0,0,0,0,4.16,2
4,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,개포동,0,0,0,0,0,0,0,4.04,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87356,45000,2004,오피스텔,66.64,8.0,서울특별시 강남구 역삼로 432,NaN,NaN,2019-07-22,19,...,대치동,2,3,1,1,1,0,0,2.64,1
87357,38000,2005,오피스텔,43.68,5.0,서울특별시 강남구 압구정로 461,NaN,NaN,2019-06-21,18,...,청담동,13,3,17,0,2,0,0,2.74,1
87358,23500,2014,오피스텔,31.57,5.0,서울특별시 강남구 도산대로 406,NaN,NaN,2019-07-20,9,...,청담동,13,3,31,1,2,0,0,2.64,0
87359,3000,2020,오피스텔,29.51,6.0,서울특별시 강남구 논현로20길 12,NaN,NaN,2020-09-08,3,...,개포동,0,1,0,0,0,0,0,2.44,1


# 전처리 

# 1. 특성공학
- 특징 구축방법(Feature Engineering)을 주로 진행 
- 이후 중복특징기법 진행


# 1-1-1. 전용면적 Feature Engineering통한 지표변수 변환
- 고유값이 5813개가 있다.
- 다른 지표와 비교가 어렵다.
- '0-30', '30-60', '60-90', '90-120', '120+'평수를 구간을 나눠 새로운 칼럼으로 만들겠다.

In [3]:
df['전용면적(㎡)'].describe()

count    87361.000000
mean        67.776567
std         37.183271
min         11.390000
25%         40.560000
50%         59.920000
75%         84.930000
max        301.470000
Name: 전용면적(㎡), dtype: float64

In [4]:
df['전용면적(㎡)'].nunique()

5813

In [5]:
bins = [0, 30, 60, 90, 120, float('inf')]
labels = ['0-30', '30-60', '60-90', '90-120', '120+']
df['전용면적band'] = pd.cut(df['전용면적(㎡)'], bins=bins, labels=labels, include_lowest=True)

In [6]:
df

,보증금(만원),건축년도,구분,전용면적(㎡),층,도로명주소_x,계약시작,계약종료,계약년월일,건축나이,...,행정구코드,스타벅스 개수_y,미쉐린 개수,맥도날드 개수,대규모점포 개수,전통시장 개수,역세권 개수,금리,유치원 개수,전용면적band
0,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,0,0,0,0,0,4.04,2,30-60
1,14333,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-10-01,2024-10-01,2022-10-05,32,...,0,0,0,0,0,0,0,4.82,2,30-60
2,17115,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,0,0,0,0,0,4.16,2,30-60
3,17640,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,0,0,0,0,0,4.16,2,30-60
4,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,0,0,0,0,0,4.04,2,30-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87356,45000,2004,오피스텔,66.64,8.0,서울특별시 강남구 역삼로 432,NaN,NaN,2019-07-22,19,...,2,3,1,1,1,0,0,2.64,1,60-90
87357,38000,2005,오피스텔,43.68,5.0,서울특별시 강남구 압구정로 461,NaN,NaN,2019-06-21,18,...,13,3,17,0,2,0,0,2.74,1,30-60
87358,23500,2014,오피스텔,31.57,5.0,서울특별시 강남구 도산대로 406,NaN,NaN,2019-07-20,9,...,13,3,31,1,2,0,0,2.64,0,30-60
87359,3000,2020,오피스텔,29.51,6.0,서울특별시 강남구 논현로20길 12,NaN,NaN,2020-09-08,3,...,0,1,0,0,0,0,0,2.44,1,0-30


# 1-1-2. 전용면적 + 행정구  Feature Engineering통한 중복 특징 변환

In [7]:
df['행정구_전용면적band'] = df['행정구'] + df['전용면적band'].astype(str)

In [8]:
df

,보증금(만원),건축년도,구분,전용면적(㎡),층,도로명주소_x,계약시작,계약종료,계약년월일,건축나이,...,스타벅스 개수_y,미쉐린 개수,맥도날드 개수,대규모점포 개수,전통시장 개수,역세권 개수,금리,유치원 개수,전용면적band,행정구_전용면적band
0,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,0,0,0,0,4.04,2,30-60,개포동30-60
1,14333,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-10-01,2024-10-01,2022-10-05,32,...,0,0,0,0,0,0,4.82,2,30-60,개포동30-60
2,17115,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,0,0,0,0,4.16,2,30-60,개포동30-60
3,17640,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,0,0,0,0,4.16,2,30-60,개포동30-60
4,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,0,0,0,0,4.04,2,30-60,개포동30-60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87356,45000,2004,오피스텔,66.64,8.0,서울특별시 강남구 역삼로 432,NaN,NaN,2019-07-22,19,...,3,1,1,1,0,0,2.64,1,60-90,대치동60-90
87357,38000,2005,오피스텔,43.68,5.0,서울특별시 강남구 압구정로 461,NaN,NaN,2019-06-21,18,...,3,17,0,2,0,0,2.74,1,30-60,청담동30-60
87358,23500,2014,오피스텔,31.57,5.0,서울특별시 강남구 도산대로 406,NaN,NaN,2019-07-20,9,...,3,31,1,2,0,0,2.64,0,30-60,청담동30-60
87359,3000,2020,오피스텔,29.51,6.0,서울특별시 강남구 논현로20길 12,NaN,NaN,2020-09-08,3,...,1,0,0,0,0,0,2.44,1,0-30,개포동0-30


# 1-2-1. 층  Feature Engineering통한 지표변수 변환
- 층 고유값은 67개이다.
- '-1-3', '3-5', '5-10', '10-67'층을 구간으로 나눠 새로운 칼럼을 만들겠다.

In [9]:
df['층'].describe()

count    87361.000000
mean         6.859915
std          5.758265
min         -1.000000
25%          3.000000
50%          5.000000
75%         10.000000
max         67.000000
Name: 층, dtype: float64

In [10]:
df['층'].nunique()

67

In [11]:

df = pd.DataFrame(df)

# Define the bin edges
bins = [-float('inf'), 3, 5, 10, float('inf')]
bin_labels = ['-1-3', '3-5', '5-10', '10-67']

# Create a new column 'band' that contains the bin labels
df['층band'] = pd.cut(df['층'], bins=bins, labels=bin_labels, right=False)

df

,보증금(만원),건축년도,구분,전용면적(㎡),층,도로명주소_x,계약시작,계약종료,계약년월일,건축나이,...,미쉐린 개수,맥도날드 개수,대규모점포 개수,전통시장 개수,역세권 개수,금리,유치원 개수,전용면적band,행정구_전용면적band,층band
0,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,0,0,0,4.04,2,30-60,개포동30-60,3-5
1,14333,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-10-01,2024-10-01,2022-10-05,32,...,0,0,0,0,0,4.82,2,30-60,개포동30-60,3-5
2,17115,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,0,0,0,4.16,2,30-60,개포동30-60,3-5
3,17640,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,0,0,0,4.16,2,30-60,개포동30-60,3-5
4,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,0,0,0,4.04,2,30-60,개포동30-60,3-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87356,45000,2004,오피스텔,66.64,8.0,서울특별시 강남구 역삼로 432,NaN,NaN,2019-07-22,19,...,1,1,1,0,0,2.64,1,60-90,대치동60-90,5-10
87357,38000,2005,오피스텔,43.68,5.0,서울특별시 강남구 압구정로 461,NaN,NaN,2019-06-21,18,...,17,0,2,0,0,2.74,1,30-60,청담동30-60,5-10
87358,23500,2014,오피스텔,31.57,5.0,서울특별시 강남구 도산대로 406,NaN,NaN,2019-07-20,9,...,31,1,2,0,0,2.64,0,30-60,청담동30-60,5-10
87359,3000,2020,오피스텔,29.51,6.0,서울특별시 강남구 논현로20길 12,NaN,NaN,2020-09-08,3,...,0,0,0,0,0,2.44,1,0-30,개포동0-30,5-10


# 1-3-2. 층+ 전용면적 Feature Engineering통한 중복 특징 변환

In [12]:
df['층band_전용면적band'] = df['층band'].astype(str) + '/' + df['전용면적band'].astype(str)


In [13]:
df['층band_전용면적band'].nunique()

20

# 1-3-1. 계약년월일 Feature Engineering통한 지표변수 변환
- 계약년월일도 고유값이 1821개가 넘는다.
- 년월일을 각년도마다의 1~4분기로 나눠 구간을 만들어보았다.

In [14]:
df['계약년월일'].describe()

count          87361
unique          1821
top       2022-05-11
freq             155
Name: 계약년월일, dtype: object

In [15]:
df['계약년월일'].nunique()

1821

In [16]:
date_range = pd.date_range(start='2018-01-01', end='2022-12-31', freq='D')
data = {'계약년월일': date_range}


# '계약년월일' 열을 날짜 형식으로 변환
df['계약년월일'] = pd.to_datetime(df['계약년월일'])

# 분기를 나타내는 새로운 열 생성
df['분기'] = df['계약년월일'].dt.to_period('Q')

df

,보증금(만원),건축년도,구분,전용면적(㎡),층,도로명주소_x,계약시작,계약종료,계약년월일,건축나이,...,대규모점포 개수,전통시장 개수,역세권 개수,금리,유치원 개수,전용면적band,행정구_전용면적band,층band,층band_전용면적band,분기
0,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,0,4.04,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q2
1,14333,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-10-01,2024-10-01,2022-10-05,32,...,0,0,0,4.82,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q4
2,17115,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,0,4.16,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q3
3,17640,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,0,4.16,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q3
4,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,0,4.04,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87356,45000,2004,오피스텔,66.64,8.0,서울특별시 강남구 역삼로 432,NaN,NaN,2019-07-22,19,...,1,0,0,2.64,1,60-90,대치동60-90,5-10,5-10/60-90,2019Q3
87357,38000,2005,오피스텔,43.68,5.0,서울특별시 강남구 압구정로 461,NaN,NaN,2019-06-21,18,...,2,0,0,2.74,1,30-60,청담동30-60,5-10,5-10/30-60,2019Q2
87358,23500,2014,오피스텔,31.57,5.0,서울특별시 강남구 도산대로 406,NaN,NaN,2019-07-20,9,...,2,0,0,2.64,0,30-60,청담동30-60,5-10,5-10/30-60,2019Q3
87359,3000,2020,오피스텔,29.51,6.0,서울특별시 강남구 논현로20길 12,NaN,NaN,2020-09-08,3,...,0,0,0,2.44,1,0-30,개포동0-30,5-10,5-10/0-30,2020Q3


In [17]:
df['분기'].nunique()

20

# 1-4. 건축나이 Feature Engineering통한 대표 특징 변환
- 건축나이도 0살부터(2022년)~ 51살(1971년)까지 있음 
- [수명분석을 통한 건축설비의 내용연수 결정방안에 관한 연구] 논문참고하여 0-10 청년기, 10-20 장년기, 20-30 중년기, 30-40 노년기, 50+(고령기)로 구분band

In [18]:
df['건축나이'].describe()

count    87361.000000
mean        22.508099
std         12.485836
min          0.000000
25%         10.000000
50%         21.000000
75%         31.000000
max         51.000000
Name: 건축나이, dtype: float64

In [19]:
df['건축나이'].nunique()

52

In [20]:

# 구간 정의
bins = [0, 10, 20, 30, 40, 50, float('inf')]  # 'inf'는 무한대를 나타냄

# 구간별 레이블
labels = ['0-10(청년기)', '10-20(장년기)', '20-30(중년기)', '30-40(중년기)', '40-50(노년기)', '50+(고령기)']

# 구간별로 '건축수명band' 컬럼 추가
df['건축수명band'] = pd.cut(df['건축나이'], bins=bins, labels=labels, right=False)

df

,보증금(만원),건축년도,구분,전용면적(㎡),층,도로명주소_x,계약시작,계약종료,계약년월일,건축나이,...,전통시장 개수,역세권 개수,금리,유치원 개수,전용면적band,행정구_전용면적band,층band,층band_전용면적band,분기,건축수명band
0,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,4.04,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q2,30-40(중년기)
1,14333,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-10-01,2024-10-01,2022-10-05,32,...,0,0,4.82,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q4,30-40(중년기)
2,17115,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,4.16,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q3,30-40(중년기)
3,17640,1991,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-07-01,2024-07-01,2022-07-07,32,...,0,0,4.16,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q3,30-40(중년기)
4,7500,1989,단독다가구,44.00,3.0,서울특별시 강남구 논현로4길,2022-06-01,2024-06-01,2022-06-01,34,...,0,0,4.04,2,30-60,개포동30-60,3-5,3-5/30-60,2022Q2,30-40(중년기)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87356,45000,2004,오피스텔,66.64,8.0,서울특별시 강남구 역삼로 432,NaN,NaN,2019-07-22,19,...,0,0,2.64,1,60-90,대치동60-90,5-10,5-10/60-90,2019Q3,10-20(장년기)
87357,38000,2005,오피스텔,43.68,5.0,서울특별시 강남구 압구정로 461,NaN,NaN,2019-06-21,18,...,0,0,2.74,1,30-60,청담동30-60,5-10,5-10/30-60,2019Q2,10-20(장년기)
87358,23500,2014,오피스텔,31.57,5.0,서울특별시 강남구 도산대로 406,NaN,NaN,2019-07-20,9,...,0,0,2.64,0,30-60,청담동30-60,5-10,5-10/30-60,2019Q3,0-10(청년기)
87359,3000,2020,오피스텔,29.51,6.0,서울특별시 강남구 논현로20길 12,NaN,NaN,2020-09-08,3,...,0,0,2.44,1,0-30,개포동0-30,5-10,5-10/0-30,2020Q3,0-10(청년기)


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87361 entries, 0 to 87360
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   보증금(만원)         87361 non-null  int64         
 1   건축년도            87361 non-null  int64         
 2   구분              87361 non-null  object        
 3   전용면적(㎡)         87361 non-null  float64       
 4   층               87361 non-null  float64       
 5   도로명주소_x         87361 non-null  object        
 6   계약시작            25069 non-null  object        
 7   계약종료            25069 non-null  object        
 8   계약년월일           87361 non-null  datetime64[ns]
 9   건축나이            87361 non-null  int64         
 10  Latitude        87361 non-null  float64       
 11  Longitude       87361 non-null  float64       
 12  행정구             87361 non-null  object        
 13  행정구코드           87361 non-null  int64         
 14  스타벅스 개수_y       87361 non-null  int64         
 15  미쉐

# 1-5. 구분 Feature Engineering통한 중복 특성 변환
- 구분은 4가지의 고유값이 있다.
- 더이상 나눌것은 없고 구분, 층band, 전용면적band를 활용하여 중복특성으로 변환하겠다.

In [22]:
df['구분'] 

0        단독다가구
1        단독다가구
2        단독다가구
3        단독다가구
4        단독다가구
         ...  
87356     오피스텔
87357     오피스텔
87358     오피스텔
87359     오피스텔
87360     오피스텔
Name: 구분, Length: 87361, dtype: object

In [23]:
df['구분'].describe()

count     87361
unique        4
top         아파트
freq      53963
Name: 구분, dtype: object

In [24]:
df['구분'].nunique()

4

In [25]:
df['구분_층band_전용면적band'] = df['구분'] +'/' + df['층band'].astype(str) + '/' + df['전용면적band'].astype(str)

In [30]:
df['구분_층band_전용면적band']

0        단독다가구/3-5/30-60
1        단독다가구/3-5/30-60
2        단독다가구/3-5/30-60
3        단독다가구/3-5/30-60
4        단독다가구/3-5/30-60
              ...       
87356    오피스텔/5-10/60-90
87357    오피스텔/5-10/30-60
87358    오피스텔/5-10/30-60
87359     오피스텔/5-10/0-30
87360     오피스텔/5-10/0-30
Name: 구분_층band_전용면적band, Length: 87361, dtype: object

In [31]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# 평균 보증금 그룹화
grouped_deposit = df.groupby('구분_층band_전용면적band')['보증금(만원)'].mean()

# 평균 금리 그룹화 
grouped_interest = df.groupby('구분_층band_전용면적band')['금리'].mean()

# 그래프 생성
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=grouped_deposit.index, y=grouped_deposit.values, name='평균 보증금(만원)'))
fig.add_trace(go.Scatter(x=grouped_interest.index, y=grouped_interest.values, mode='lines+markers', name='금리'), secondary_y=True)

# 레이아웃 설정
fig.update_layout(title='평균 보증금과 금리', xaxis_tickangle=-45)

# 축 설정
fig.update_xaxes(title_text='구분_층band_전용면적band')

# 보증금 축 설정
fig.update_yaxes(title_text='평균 보증금(만원)', secondary_y=False)

# 금리 축 설정
fig.update_yaxes(title_text='금리', secondary_y=True, range=[0, 4])  # 금리 값 범위 설정 (0 ~ 10)

fig.show()


# 1-5. 계절 

# 1-5. 대통령 재임기간

# 문제해결!

# 2-1 정권교체 시기가 전세 보증금에 영향을 주는가?
문재인정권에서 윤석렬정권 아파트데이터 수집 비교.
- 금리 비교
- 보증금 그래프도 추가
- 부동산정책도 비교

# 2-2. 스타벅스, 맥도날드, 미쉐린, 백화점, 코엑스 바로 옆(100m, 200m) 바로옆에 있는 아파트들이 더 비쌀까? 
- 최대한 한 이미지에 나오고 싶다.


# 2-2.  계절의 변화동안에 보증금이 변화는 어떨까?

# 2-3. 강남구에서 평균 보증금이 가장 높은 top5는 무엇일까?
- 주요요인 분석(도메인 지식 사용)
- 계절

# 2-4. 